<a href="https://colab.research.google.com/github/pxpc2/qualvinho/blob/main/decisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.collab import *
from fastai.tabular.all import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.2 MB/s eta 0:00:00
Mounted at /content/gdrive


In [30]:
wines = pd.read_csv('/content/gdrive/MyDrive/XWines_Slim/XWines_Slim_1K_wines.csv',
                    usecols=['WineID', 'WineName', 'Type', 'Grapes',
                             'Body', 'Harmonize', 'Acidity', 'ABV', 'Code', 'Country', 'Elaborate', 'WineryName'],
                     delimiter=',', header=None, skiprows=1, low_memory=False,
                    names=['WineID', 'WineName', 'Type', 'Elaborate', 'Grapes',
                           'Harmonize', 'ABV', 'Body', 'Acidity', 'Code',
                           'Country', 'RegionID', 'RegionName', 'WineryID',
                           'WineryName','Website', 'Vintages'],
                    dtype={'WineID': int, 'WineName': str, 'Type': str,
                           'Grapes': str, 'Harmonize': str, 'Body': str,
                           'Acidity': str, 'ABV': str,
                           'Code': str, 'Elaborate': str, 'Country': str, 'WineryName': str})



ratings = pd.read_csv('/content/gdrive/MyDrive/XWines_Slim/XWines_Slim_150K_ratings.csv',
                      usecols=['UserID', 'WineID', 'Rating'],
                      delimiter=',', header=None, skiprows=1, low_memory=False,
                      names=['RatingID', 'UserID', 'WineID', 'Vintage', 'Rating', 'Date'],
                      dtype={'UserID': int, 'WineID': int, 'Rating': float})

In [31]:
df = ratings.merge(wines)
df.columns

Index(['UserID', 'WineID', 'Rating', 'WineName', 'Type', 'Elaborate', 'Grapes',
       'Harmonize', 'ABV', 'Body', 'Acidity', 'Code', 'Country', 'WineryName'],
      dtype='object')

In [10]:
df['Body'].unique()

array(['Very full-bodied', 'Full-bodied', 'Medium-bodied', 'Light-bodied', 'Very light-bodied'], dtype=object)

In [17]:
df['Rating'].unique()

array([4.5, 4. , 3.5, 5. , 3. , 1. , 2. , 2.5, 1.5])

In [19]:
dep_var = 'Rating'

In [20]:
procs = [Categorify, FillMissing]

In [32]:
splits = RandomSplitter(seed=42)(df)

In [33]:
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

In [34]:
len(to.train),len(to.valid)

(120000, 30000)

In [35]:
to.show(3)

,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,WineryName,UserID,WineID,Rating
137355,Unwooded Pinot Noir,Red,Varietal/100%,['Pinot Noir'],"['Beef', 'Veal', 'Game Meat', 'Poultry']",12.5,Medium-bodied,High,ZA,South Africa,Haute Cabrière,1149843,171438,3.5
35229,Hemisferio Reserva Carmenère,Red,Varietal/100%,['Carmenère'],"['Beef', 'Pasta', 'Lamb', 'Poultry']",13.0,Full-bodied,High,CL,Chile,Miguel Torres Chile,1200644,162756,2.5
96428,Malbec,Red,Varietal/100%,['Malbec'],"['Beef', 'Lamb', 'Poultry']",13.9,Very full-bodied,High,AR,Argentina,La Linda,1284340,167425,3.0


In [36]:
to.items.head(3)

,UserID,WineID,Rating,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,WineryName
137355,1149843,171438,3.5,751,3,12,194,80,15,3,1,31,24,372
35229,1200644,162756,2.5,348,3,12,60,63,20,1,1,7,6,511
96428,1284340,167425,3.0,423,3,12,134,43,29,4,1,1,1,434
